< 재구매 유형별 구매시점 예측 >
---
과제1
  1. '대차'와 '추가구매'를 분리/추정하는 로직개발  
     -> 보유 종료가 다음차량 출고보다 빠르거나 늦지만 기간 차이가 많이 나지 않으면 '대차', 많이 늦거나 없으면 '추가구매'  
     -> 자동차 등록령 제31조 제1항에 근거, 대차/추가구매 구분 기간은 기존 차량 보유 종료 일자가 신차 보유 시작일 이후 최대 91일(3개월)까지를 기준으로 지정
         제31조(말소등록 신청) ① 말소등록은 그 사유가 발생한 날부터 1개월(상속의 경우에는 상속개시일부터 3개월) 이내에 자동차등록증, 등록번호판 및 봉인을 반납하고 말소등록 사유를 증명하는 서류를 첨부하여 등록관청에 신청하여야 한다.  
     
     말소 등록 사유
         1. 자동차 해체 재활용 업자에게 폐차를 요청한 경우(규제「자동차관리법」 제13조제1항제1호)
         2. 자동차 제작·판매자 등에게 반품한 경우(규제「자동차관리법」 제13조제1항제2호)
         3. 「여객자동차 운수사업법」에 따른 차령이 초과한 경우(규제「자동차관리법」 제13조제1항제3호)
         4. 「여객자동차 운수사업법」및「화물자동차 운수사업법」에 따라 면허·등록·인가 또는 신고가 실효되거나 취소된 경우(규제「자동차관리법」 제13조제1항제4호)
         5. 천재지변·교통사고 또는 화재로 본래 기능을 회복할 수 없거나 멸실된 경우(규제「자동차관리법」 제13조제1항제5호)
         6. 자동차를 수출하는 경우(규제「자동차관리법」 제13조제1항제6호)
         
  2. 구분 결과에 대해 각각 EDA를 수행하여 고객과 차량 관점에서 특징/차이점 도출  
  > - '대차'와 '추가구매'의 분리/추정 방식은 자율.  
  > - 단, 논리전개 과정과 근거를 반드시 기재할 것.
  ---

과제2
  1. 고객별 6개월 내 '대차' 또는 '추가구매' 가능성(Score)에 대한 예측모델 개발
  > - '고객별' 예측모델로, '대차' 또는 '추가구매' 중 1개를 선택하여 모델링 가능.  
  > - 과제1의 유형 분리 알고리즘을 활용한 종속변수 생성.  
  > - 주어진 데이터를 활용한 고객별 독립변수 생성.

# 데이터 불러오기

In [1]:
import pandas as pd
from datetime import timedelta

In [2]:
# 원본데이터 불러올 코드
cus = pd.read_csv("'21년 해카톤_고객정보.csv", encoding='cp949')    # 상대경로 지정
car = pd.read_csv("'21년 해카톤_차량정보.csv", encoding='cp949')
cntc = pd.read_csv("'21년 해카톤_접촉정보.csv", encoding='cp949')

In [3]:
### cus(customer) : 고객정보 데이터프레임
print(cus.shape)
print('변수명 : 개인사업자여부, 성별, 생년월, 주소(행정시도명), 주소(시군구), 주택 평형, 주택 가격(단위:만원)')
cus.head(5)    # 고객ID, 개인사업자여부, 성별, 생년월, 주소(행정시도명), 주소(시군구), 주소(읍면동), 주택 평형, 주택 평균가격

(1096206, 9)
변수명 : 개인사업자여부, 성별, 생년월, 주소(행정시도명), 주소(시군구), 주택 평형, 주택 가격(단위:만원)


,CUS_ID,PSN_BIZR_YN,SEX_SCN_NM,TYMD,CUS_ADM_TRY_NM,CUS_N_ADMZ_NM,CUS_ADMB_NM,CLB_HOUS_PYG_NM,REAI_BZTC_AVG_PCE
0,AOZZZZO010000035,NaN,남자,19300300,전라남도,영암군,시종면,NaN,NaN
1,AOZZZZO010000067,NaN,남자,19330600,경상남도,진주시,NaN,NaN,NaN
2,AOZZZZO010000190,NaN,남자,19370700,NaN,NaN,NaN,23,13400.0
3,AOZZZZO010000371,NaN,남자,19400800,서울특별시,마포구,NaN,NaN,NaN
4,AOZZZZO010000492,NaN,남자,19411100,광주광역시,북구,NaN,NaN,NaN


In [4]:
### car : 차량정보 데이터프레임
print(car.shape)
print('변수명 : 차량ID, 고객ID, 출고일자, 보유시작일자, 보유종료일자, 차명, 차량등급명1, 차량등급명2, 엔진타입명, 트림명')
car.head(5)    # 차량ID, 고객ID, 출고일자, 보유시작일자, 보유종료일자, 차명, 차량등급명1, 차량등급명2, 엔진타입명, 트림명

(1835830, 10)
변수명 : 차량ID, 고객ID, 출고일자, 보유시작일자, 보유종료일자, 차명, 차량등급명1, 차량등급명2, 엔진타입명, 트림명


,CAR_ID,CUS_ID,WHOT_DT,CAR_HLDG_STRT_DT,CAR_HLDG_FNH_DT,CAR_NM,CAR_CGRD_NM_1,CAR_CGRD_NM_2,CAR_ENG_NM,CAR_TRIM_NM
0,H1308052741CBFA022907,AOZZZZO010000035,20140627,20140627,20170831.0,LF쏘나타,승용,중형,가솔린 2.0 CVVL,SMART
1,H1308050341LBEA656859,AOZZZZO010000067,20140711,20140711,NaN,YF쏘나타,승용,중형,LPG 2.0,PREMIUM
2,H1308052741DBFA038433,AOZZZZO010000190,20140707,20140707,NaN,LF쏘나타,승용,중형,LPi 2.0,PREMIUM
3,H13080607413BFA007670,AOZZZZO010000371,20140711,20140711,NaN,그랜저HG,승용,대형,하이브리드,PREMIUM
4,H1308052741DBFA091812,AOZZZZO010000492,20150120,20150120,NaN,LF쏘나타,승용,중형,LPi 2.0,PREMIUM


In [5]:
### cntc(contect) : 접촉정보 데이터프레임
print(cntc.shape)
print('변수명 : 접촉일련번호, 고객ID, 접촉일자, 접촉채널명, 접촉업무명')
cntc.head(5)    # 접촉일련번호, 고객ID, 접촉일자, 접촉채널명, 접촉업무명

(17734097, 5)
변수명 : 접촉일련번호, 고객ID, 접촉일자, 접촉채널명, 접촉업무명


,CNTC_SN,CUS_ID,CNTC_DT,CNTC_CHAN_NM,CNTC_AFFR_SCN_NM
0,172591627,AOZZZZO010000035,20150504,방문(대면),정비
1,172610507,AOZZZZO010000035,20150504,방문(대면),정비
2,306638131,AOZZZZO010000035,20161202,방문(대면),정비
3,306641393,AOZZZZO010000035,20161202,방문(대면),정비
4,344010325,AOZZZZO010000035,20170215,방문(대면),정비


# 데이터 전처리

---
---
---
## 고객정보 전처리
---
---
---

In [6]:
### cus_df : 고객정보 데이터 복사
cus_df = cus.copy()

### [고객정보] 개인 사업자 여부 결측치

In [7]:
# 개인 사업자 여부 결측값 처리
print(cus_df.PSN_BIZR_YN.unique())                              # 컬럼 내 유니크 값 확인
cus_df['PSN_BIZR_YN'] = cus_df['PSN_BIZR_YN'].fillna('N')
cus_df['PSN_BIZR_YN'].head()

[nan 'Y']


0    N
1    N
2    N
3    N
4    N
Name: PSN_BIZR_YN, dtype: object

### [고객정보] 고객 나이 생성

In [8]:
cus_df['TYMD'] = cus_df['TYMD'].astype(str)
cus_df['TYMD'] = cus_df['TYMD'].replace('99991231','19001231')
cus_df['TYMD'] = cus_df['TYMD'].str[:7]
cus_df['TYMD'] = cus_df['TYMD'].str.pad(8, side='right', fillchar='1')

In [9]:
cus_df['CUS_AGE'] = ''            # 고객별 나이 컬럼
today = pd.datetime.now().year    # 현재 시점의 해를 기준

for i in range(len(cus_df)):
    birth = pd.to_datetime(cus_df['TYMD'][i]).year
    cus_df['CUS_AGE'][i] = today-birth

<ipython-input-9-cf4f4d2cd20b>:2: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  today = pd.datetime.now().year    # 현재 시점의 해를 기준
<ipython-input-9-cf4f4d2cd20b>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cus_df['CUS_AGE'][i] = today-birth


In [10]:
# 고객 나이 컬럼 위치 재배치
cus_df = cus_df[['CUS_ID','PSN_BIZR_YN','SEX_SCN_NM','TYMD','CUS_AGE','CUS_ADM_TRY_NM','CUS_N_ADMZ_NM','CUS_ADMB_NM','CLB_HOUS_PYG_NM','REAI_BZTC_AVG_PCE']]

In [11]:
cus_df.head()

,CUS_ID,PSN_BIZR_YN,SEX_SCN_NM,TYMD,CUS_AGE,CUS_ADM_TRY_NM,CUS_N_ADMZ_NM,CUS_ADMB_NM,CLB_HOUS_PYG_NM,REAI_BZTC_AVG_PCE
0,AOZZZZO010000035,N,남자,19300301,91,전라남도,영암군,시종면,NaN,NaN
1,AOZZZZO010000067,N,남자,19330601,88,경상남도,진주시,NaN,NaN,NaN
2,AOZZZZO010000190,N,남자,19370701,84,NaN,NaN,NaN,23,13400.0
3,AOZZZZO010000371,N,남자,19400801,81,서울특별시,마포구,NaN,NaN,NaN
4,AOZZZZO010000492,N,남자,19411101,80,광주광역시,북구,NaN,NaN,NaN


### [고객정보] 고객 거주 지역 전처리 1

In [12]:
# 광역자치단체 지역 결측치 처리(평당 평균 가격 계산 이후 결측치 추가 처리 예정)
print(cus_df.CUS_ADM_TRY_NM.unique())
cus_df['CUS_ADM_TRY_NM'] = cus_df['CUS_ADM_TRY_NM'].fillna('NA')
cus_df['CUS_ADM_TRY_NM'].head()

['전라남도' '경상남도' nan '서울특별시' '광주광역시' '경기도' '전라북도' '충청북도' '제주특별자치도' '경상북도'
 '세종특별자치시' '부산광역시' '대전광역시' '강원도' '인천광역시' '대구광역시' '충청남도' '울산광역시']


0     전라남도
1     경상남도
2       NA
3    서울특별시
4    광주광역시
Name: CUS_ADM_TRY_NM, dtype: object

In [13]:
# 지역별 시/군/구 구분
print(cus_df.CUS_N_ADMZ_NM.unique())
# 시군구로 끝나지 않는 데이터 필터링 및 수정
#ft = cus_anls[cus_anls['CUS_N_ADMZ_NM'].str.endswith(('시','군','구')) == False]     시군구로 끝나지 않는 지역 확인 코드
cus_df['CUS_N_ADMZ_NM'] = cus_df['CUS_N_ADMZ_NM'].replace(['고양시 일산서','고양시 일산동','창원시 마산회','창원시 마산합'],
                                                              ['고양시','고양시','창원시','창원시'])
cus_df['CUS_N_ADMZ_NM'] = cus_df['CUS_N_ADMZ_NM'].str[-1]
cus_df['CUS_N_ADMZ_NM'] = cus_df['CUS_N_ADMZ_NM'].fillna('NA')
cus_df['CUS_N_ADMZ_NM'].head()

['영암군' '진주시' nan '마포구' '북구' '양산시' '부천시' '군산시' '수원시 팔달구' '군포시' '중구' '합천군'
 '함안군' '산청군' '영동군' '제주시' '구미시' '양주시' '동래구' '경주시' '은평구' '청주시 서원구' '금정구'
 '연제구' '평택시' '성남시 수정구' '성남시 중원구' '광산구' '남양주시' '서귀포시' '광진구' '해운대구'
 '수원시 영통구' '성동구' '강서구' '홍천군' '고양시 덕양구' '여수시' '구로구' '동작구' '포항시 북구' '계양구'
 '속초시' '용인시 처인구' '안산시 상록구' '달성군' '삼척시' '충주시' '의정부시' '대덕구' '수원시 장안구' '강동구'
 '수성구' '청양군' '유성구' '부평구' '성남시 분당구' '김포시' '고양시 일산서' '고양시 일산동' '상주시' '오산시'
 '동구' '용인시 기흥구' '남구' '창원시 진해구' '화성시' '서천군' '춘천시' '청주시 상당구' '안양시 동안구' '서구'
 '달서구' '예산군' '시흥시' '강남구' '노원구' '장흥군' '함양군' '과천시' '무안군' '전주시 덕진구' '고흥군'
 '청주시 청원구' '경산시' '서산시' '도봉구' '당진시' '울주군' '파주시' '창원시 의창구' '천안시 동남구' '아산시'
 '영등포구' '송파구' '연수구' '양천구' '의왕시' '창원시 성산구' '계룡시' '수원시 권선구' '정선군' '서대문구'
 '청주시 흥덕구' '태안군' '안성시' '중랑구' '태백시' '창원시 마산회' '미추홀구' '광주시' '거제시' '창원시 마산합'
 '김해시' '원주시' '동대문구' '수영구' '제천시' '하동군' '기장군' '의성군' '청도군' '전주시 완산구' '보은군'
 '영덕군' '목포시' '남해군' '영양군' '강릉시' '평창군' '화천군' '공주시' '영주시' '화순군' '광명시' '부산진구'
 '익산시' '논산시' '서초구' '김천시' '인제군' '칠곡군' '영도구' '통영시' '사하구' '관악구' '

0     군
1     시
2    NA
3     구
4     구
Name: CUS_N_ADMZ_NM, dtype: object

### [고객정보] 주택평균가격 결측치 처리

In [14]:
# 평수 데이터에서 문자 제외하고 숫자만 추출하기 위한 정규표현식
cus_df['CLB_HOUS_PYG_NM'] = cus_df['CLB_HOUS_PYG_NM'].str.extract(r'(\d*)')
# 'x' 시/도의 주택 평형, 가격이 null이 아닌 데이터 추출
avg_pp = cus_df[cus_df['CLB_HOUS_PYG_NM'].notnull()]                          ### avg_pp : 시도별 평당 평균 가격 dataframe
avg_pp = avg_pp[avg_pp['REAI_BZTC_AVG_PCE'].notnull()].reset_index(drop=True)
avg_pp = avg_pp[['CUS_ADM_TRY_NM','CLB_HOUS_PYG_NM','REAI_BZTC_AVG_PCE']]
# 형 변환
avg_pp['CLB_HOUS_PYG_NM'] = avg_pp['CLB_HOUS_PYG_NM'].astype(int)
avg_pp['REAI_BZTC_AVG_PCE'] = avg_pp['REAI_BZTC_AVG_PCE'].astype(int)

In [15]:
# 주택 평균 가격 계산을 위한 계산
avg_pp = avg_pp.groupby(avg_pp['CUS_ADM_TRY_NM']).sum()
avg_pp['PYG_AVG_PCE'] = round(avg_pp['REAI_BZTC_AVG_PCE']/avg_pp['CLB_HOUS_PYG_NM']).astype(int)
avg_pp = avg_pp['PYG_AVG_PCE'].reset_index()
avg_pp = avg_pp.drop(avg_pp[avg_pp['CUS_ADM_TRY_NM']=='NA'].index)
avg_pp = avg_pp.sort_values(by=['PYG_AVG_PCE'], ascending=False).reset_index(drop=True)
avg_pp

,CUS_ADM_TRY_NM,PYG_AVG_PCE
0,서울특별시,3367
1,세종특별자치시,2020
2,경기도,1502
3,대전광역시,1253
4,대구광역시,1244
5,부산광역시,1209
6,인천광역시,1108
7,제주특별자치도,1069
8,울산광역시,972
9,광주광역시,907


In [16]:
cus_df = pd.merge(cus_df, avg_pp, how='left', on='CUS_ADM_TRY_NM')
# 주택 평형 결측치 처리를 위해 고객 주택 평수 중앙값 확인
print(cus_df.median())

TYMD                 19680301.0
CUS_AGE                    53.0
CLB_HOUS_PYG_NM            32.0
REAI_BZTC_AVG_PCE       31900.0
PYG_AVG_PCE              1209.0
dtype: float64


In [17]:
print(cus_df.CLB_HOUS_PYG_NM.unique())
# 중앙값 '32'로 결측치 처리 및 정수형으로 변환
cus_df['CLB_HOUS_PYG_NM'] = cus_df['CLB_HOUS_PYG_NM'].fillna(32)
#cus_anls['CLB_HOUS_PYG_NM'] = pd.to_numeric(cus_anls['CLB_HOUS_PYG_NM'], errors='ignore')
cus_df['CLB_HOUS_PYG_NM'] = cus_df['CLB_HOUS_PYG_NM'].astype(int)
cus_df.head()

[nan '23' '50' '42' '20' '24' '14' '52' '32' '35' '27' '28' '44' '34' '33'
 '51' '37' '18' '26' '36' '15' '49' '46' '31' '40' '22' '60' '16' '67'
 '29' '30' '17' '38' '48' '25' '59' '19' '43' '39' '13' '55' '9' '45' '21'
 '41' '53' '56' '12' '10' '61' '63' '72' '57' '47' '66' '65' '58' '64'
 '11' '54' '81' '69' '62' '87' '92' '75' '70' '114' '8' '110' '84' '7'
 '74' '71' '90' '68' '85' '88' '73' '77' '6' '150' '94' '76' '86' '117'
 '99' '80' '101' '79' '78' '116' '83' '89' '93' '82' '100' '95' '104'
 '107' '299' '98' '103' '5' '96' '91' '337' '186' '102' '97' '105' '124'
 '112' '144' '115' '4' '176' '3' '180' '164' '152' '133' '129' '0' '324'
 '108' '109' '111']


,CUS_ID,PSN_BIZR_YN,SEX_SCN_NM,TYMD,CUS_AGE,CUS_ADM_TRY_NM,CUS_N_ADMZ_NM,CUS_ADMB_NM,CLB_HOUS_PYG_NM,REAI_BZTC_AVG_PCE,PYG_AVG_PCE
0,AOZZZZO010000035,N,남자,19300301,91,전라남도,군,시종면,32,NaN,631.0
1,AOZZZZO010000067,N,남자,19330601,88,경상남도,시,NaN,32,NaN,686.0
2,AOZZZZO010000190,N,남자,19370701,84,NA,NA,NaN,23,13400.0,NaN
3,AOZZZZO010000371,N,남자,19400801,81,서울특별시,구,NaN,32,NaN,3367.0
4,AOZZZZO010000492,N,남자,19411101,80,광주광역시,구,NaN,32,NaN,907.0


In [18]:
# 고객 주택 가격, 지역 평당 가격 결측치 처리
cus_df['REAI_BZTC_AVG_PCE'] = cus_df['REAI_BZTC_AVG_PCE'].fillna(0)
cus_df['PYG_AVG_PCE'] = cus_df['PYG_AVG_PCE'].fillna(0)

for i in range (len(cus_df)):
    if cus_df['REAI_BZTC_AVG_PCE'][i] == 0:
        cus_df['REAI_BZTC_AVG_PCE'][i] = cus_df['CLB_HOUS_PYG_NM'][i]*cus_df['PYG_AVG_PCE'][i]
    else :
        pass

<ipython-input-18-01a0937e00f3>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cus_df['REAI_BZTC_AVG_PCE'][i] = cus_df['CLB_HOUS_PYG_NM'][i]*cus_df['PYG_AVG_PCE'][i]


In [19]:
cus_df.head()

,CUS_ID,PSN_BIZR_YN,SEX_SCN_NM,TYMD,CUS_AGE,CUS_ADM_TRY_NM,CUS_N_ADMZ_NM,CUS_ADMB_NM,CLB_HOUS_PYG_NM,REAI_BZTC_AVG_PCE,PYG_AVG_PCE
0,AOZZZZO010000035,N,남자,19300301,91,전라남도,군,시종면,32,20192.0,631.0
1,AOZZZZO010000067,N,남자,19330601,88,경상남도,시,NaN,32,21952.0,686.0
2,AOZZZZO010000190,N,남자,19370701,84,NA,NA,NaN,23,13400.0,0.0
3,AOZZZZO010000371,N,남자,19400801,81,서울특별시,구,NaN,32,107744.0,3367.0
4,AOZZZZO010000492,N,남자,19411101,80,광주광역시,구,NaN,32,29024.0,907.0


### [고객정보] 고객 거주 지역 전처리 2

In [20]:
# NA값 확인
print(cus_df['CUS_ADM_TRY_NM'].unique())
print(cus_df['CLB_HOUS_PYG_NM'].unique())
print(cus_df['REAI_BZTC_AVG_PCE'].unique())

['전라남도' '경상남도' 'NA' '서울특별시' '광주광역시' '경기도' '전라북도' '충청북도' '제주특별자치도' '경상북도'
 '세종특별자치시' '부산광역시' '대전광역시' '강원도' '인천광역시' '대구광역시' '충청남도' '울산광역시']
[ 32  23  50  42  20  24  14  52  35  27  28  44  34  33  51  37  18  26
  36  15  49  46  31  40  22  60  16  67  29  30  17  38  48  25  59  19
  43  39  13  55   9  45  21  41  53  56  12  10  61  63  72  57  47  66
  65  58  64  11  54  81  69  62  87  92  75  70 114   8 110  84   7  74
  71  90  68  85  88  73  77   6 150  94  76  86 117  99  80 101  79  78
 116  83  89  93  82 100  95 104 107 299  98 103   5  96  91 337 186 102
  97 105 124 112 144 115   4 176   3 180 164 152 133 129   0 324 108 109
 111]
[ 20192.  21952.  13400. ...  42104. 327400.   8771.]


In [21]:
# 각 자치단체 평당 평균 가격 간 중간값을 구해서 고객 주
print(avg_pp)

list_p = []
for i in range(len(avg_pp)):
    if i == 16:
        break
    else :
        x = (avg_pp['PYG_AVG_PCE'][i]-avg_pp['PYG_AVG_PCE'][i+1])/2
        list_p.append(x)

print(list_p)

   CUS_ADM_TRY_NM  PYG_AVG_PCE
0           서울특별시         3367
1         세종특별자치시         2020
2             경기도         1502
3           대전광역시         1253
4           대구광역시         1244
5           부산광역시         1209
6           인천광역시         1108
7         제주특별자치도         1069
8           울산광역시          972
9           광주광역시          907
10           경상남도          686
11           충청남도          686
12           충청북도          645
13           전라남도          631
14           전라북도          595
15            강원도          594
16           경상북도          563
[673.5, 259.0, 124.5, 4.5, 17.5, 50.5, 19.5, 48.5, 32.5, 110.5, 0.0, 20.5, 7.0, 18.0, 0.5, 15.5]


In [22]:
for i in range(len(cus_df)):
    if cus_df['CUS_ADM_TRY_NM'][i]=='NA':
        if cus_df['REAI_BZTC_AVG_PCE'][i]!=0:
            x = round(cus_df['REAI_BZTC_AVG_PCE'][i]/cus_df['CLB_HOUS_PYG_NM'][i])
            if x <= 1936+711 & x > 1936-222:
                cus_df['CUS_ADM_TRY_NM'][i] = '세종특별자치시'
            elif x <= 1492+222 & x > 1492-121:
                cus_df['CUS_ADM_TRY_NM'][i] = '경기도'
            elif x <= 1250+121 & x > 1250-8:
                cus_df['CUS_ADM_TRY_NM'][i] = '대구광역시'
            elif x <= 1233+9 & x > 1233-26:
                cus_df['CUS_ADM_TRY_NM'][i] = '대전광역시'        
            elif x <= 1181+26 & x > 1181-39:
                cus_df['CUS_ADM_TRY_NM'][i] = '부산광역시'
            elif x <= 1103+39 & x > 1103-22:
                cus_df['CUS_ADM_TRY_NM'][i] = '인천광역시'
            elif x <= 1059+22 & x > 1059-48:
                cus_df['CUS_ADM_TRY_NM'][i] = '제주특별자치도'
            elif x <= 963+48 & x > 963-25:
                cus_df['CUS_ADM_TRY_NM'][i] = '울산광역시'
            elif x <= 912+26 & x > 912-118:
                cus_df['CUS_ADM_TRY_NM'][i] = '광주광역시'
            elif x <= 675+119 & x > 675-9:
                cus_df['CUS_ADM_TRY_NM'][i] = '경상남도'
            elif x <= 657+9 & x > 657-17:
                cus_df['CUS_ADM_TRY_NM'][i] = '충청남도'
            elif x <= 622+18 & x > 622-6:
                cus_df['CUS_ADM_TRY_NM'][i] = '충청북도'
            elif x <= 610+6 & x > 610-14:
                cus_df['CUS_ADM_TRY_NM'][i] = '전라남도'        
            elif x <= 582+14 & x > 582-4:
                cus_df['CUS_ADM_TRY_NM'][i] = '강원도'
            elif x <= 574+4 & x > 574-13:
                cus_df['CUS_ADM_TRY_NM'][i] = '전라북도'
            elif x <= 547+14:
                cus_df['CUS_ADM_TRY_NM'][i] = '경상북도'
            else:
                cus_df['CUS_ADM_TRY_NM'][i] = '서울특별시'

<ipython-input-22-85b0dfc6b709>:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cus_df['CUS_ADM_TRY_NM'][i] = '서울특별시'
<ipython-input-22-85b0dfc6b709>:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cus_df['CUS_ADM_TRY_NM'][i] = '광주광역시'
<ipython-input-22-85b0dfc6b709>:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cus_df['CUS_ADM_TRY_NM'][i] = '경상북도'
<ipython-input-22-85b0dfc6b709>:16: SettingWithCopyWarning: 
A value is trying t

In [23]:
print(cus_df.shape)
cus_df.head()

(1096206, 11)


,CUS_ID,PSN_BIZR_YN,SEX_SCN_NM,TYMD,CUS_AGE,CUS_ADM_TRY_NM,CUS_N_ADMZ_NM,CUS_ADMB_NM,CLB_HOUS_PYG_NM,REAI_BZTC_AVG_PCE,PYG_AVG_PCE
0,AOZZZZO010000035,N,남자,19300301,91,전라남도,군,시종면,32,20192.0,631.0
1,AOZZZZO010000067,N,남자,19330601,88,경상남도,시,NaN,32,21952.0,686.0
2,AOZZZZO010000190,N,남자,19370701,84,서울특별시,NA,NaN,23,13400.0,0.0
3,AOZZZZO010000371,N,남자,19400801,81,서울특별시,구,NaN,32,107744.0,3367.0
4,AOZZZZO010000492,N,남자,19411101,80,광주광역시,구,NaN,32,29024.0,907.0


In [24]:
# 원본데이터 저장
cus_df.to_csv("'21년 해카톤_고객정보(전처리적용).csv", encoding='utf-8-sig', index=False)

---
---
---
## 접촉정보 전처리
---
---
---

In [25]:
cntc_df = cntc.copy()
cntc_df.head()

,CNTC_SN,CUS_ID,CNTC_DT,CNTC_CHAN_NM,CNTC_AFFR_SCN_NM
0,172591627,AOZZZZO010000035,20150504,방문(대면),정비
1,172610507,AOZZZZO010000035,20150504,방문(대면),정비
2,306638131,AOZZZZO010000035,20161202,방문(대면),정비
3,306641393,AOZZZZO010000035,20161202,방문(대면),정비
4,344010325,AOZZZZO010000035,20170215,방문(대면),정비


### [접촉정보] 고객 접촉 유형별 컬럼 생성

In [26]:
# 접촉 유형 파악
cntc_df['CNTC_AFFR_SCN_NM'].unique()

array(['정비', '일반상담', '비포서비스', '견적', '불만상담', '이벤트', '고객초청행사', '비포서비스&무상점검',
       '시승센터시승', '영업활동(TM)', '영업활동(대면)'], dtype=object)

In [27]:
# 고객별 접촉 유형 항목별 컬럼 생성
type_s = cntc_df[(cntc_df['CNTC_AFFR_SCN_NM'] == '정비')]                 ### type_s(service) : 정비 접촉
type_c = cntc_df[(cntc_df['CNTC_AFFR_SCN_NM'] == '일반상담')]              ### type_c(consultation) : 일반상담 접촉
type_bs = cntc_df[(cntc_df['CNTC_AFFR_SCN_NM'] == '비포서비스')]           ### type_bs(before service) : 비포서비스 접촉
type_e = cntc_df[(cntc_df['CNTC_AFFR_SCN_NM'] == '견적')]                 ### type_e(estimate) : 견적 접촉
type_cc = cntc_df[(cntc_df['CNTC_AFFR_SCN_NM'] == '불만상담')]             ### type_cc(complain consultation) : 불만상담 접촉
type_ev = cntc_df[(cntc_df['CNTC_AFFR_SCN_NM'] == '이벤트')]               ### type_ev(event) : 이벤트 접촉
type_i = cntc_df[(cntc_df['CNTC_AFFR_SCN_NM'] == '고객초청행사')]           ### type_i(invitation) : 고객초청행사 접촉
type_fs = cntc_df[(cntc_df['CNTC_AFFR_SCN_NM'] == '비포서비스&무상점검')]    ### type_fs(free service) : 비포서비스&무상점검 접촉
type_td = cntc_df[(cntc_df['CNTC_AFFR_SCN_NM'] == '시승센터시승')]          ### type_td(test drive) : 시승 접촉
type_tm = cntc_df[(cntc_df['CNTC_AFFR_SCN_NM'] == '영업활동(TM)')]         ### type_tm(telemarketing) : 영업활동(텔레마케팅) 접촉

In [28]:
# 아이디별로 각 접촉 유형 횟수를 컬럼으로 생성
cntc_s = type_s.groupby(['CUS_ID']).count()
cntc_s = cntc_s.drop(['CNTC_SN','CNTC_DT','CNTC_CHAN_NM'], axis=1)
cntc_s = cntc_s.rename(columns={'CNTC_AFFR_SCN_NM':'CNTC_SERV'})

cntc_c = type_c.groupby(['CUS_ID']).count()
cntc_c = cntc_c.drop(['CNTC_SN','CNTC_DT','CNTC_CHAN_NM'], axis=1)
cntc_c = cntc_c.rename(columns={'CNTC_AFFR_SCN_NM':'CNTC_CST'})

cntc_bs = type_bs.groupby(['CUS_ID']).count()
cntc_bs = cntc_bs.drop(['CNTC_SN','CNTC_DT','CNTC_CHAN_NM'], axis=1)
cntc_bs = cntc_bs.rename(columns={'CNTC_AFFR_SCN_NM':'CNTC_B/S'})

cntc_e = type_e.groupby(['CUS_ID']).count()
cntc_e = cntc_e.drop(['CNTC_SN','CNTC_DT','CNTC_CHAN_NM'], axis=1)
cntc_e = cntc_e.rename(columns={'CNTC_AFFR_SCN_NM':'CNTC_EST'})

cntc_cc = type_cc.groupby(['CUS_ID']).count()
cntc_cc = cntc_cc.drop(['CNTC_SN','CNTC_DT','CNTC_CHAN_NM'], axis=1)
cntc_cc = cntc_cc.rename(columns={'CNTC_AFFR_SCN_NM':'CNTC_C/C'})

cntc_ev = type_ev.groupby(['CUS_ID']).count()
cntc_ev = cntc_ev.drop(['CNTC_SN','CNTC_DT','CNTC_CHAN_NM'], axis=1)
cntc_ev = cntc_ev.rename(columns={'CNTC_AFFR_SCN_NM':'CNTC_EVT'})

cntc_i = type_i.groupby(['CUS_ID']).count()
cntc_i = cntc_i.drop(['CNTC_SN','CNTC_DT','CNTC_CHAN_NM'], axis=1)
cntc_i = cntc_i.rename(columns={'CNTC_AFFR_SCN_NM':'CNTC_INVIT'})

cntc_fs = type_fs.groupby(['CUS_ID']).count()
cntc_fs = cntc_fs.drop(['CNTC_SN','CNTC_DT','CNTC_CHAN_NM'], axis=1)
cntc_fs = cntc_fs.rename(columns={'CNTC_AFFR_SCN_NM':'CNTC_F/S'})

cntc_td = type_td.groupby(['CUS_ID']).count()
cntc_td = cntc_td.drop(['CNTC_SN','CNTC_DT','CNTC_CHAN_NM'], axis=1)
cntc_td = cntc_td.rename(columns={'CNTC_AFFR_SCN_NM':'CNTC_T/D'})

cntc_tm = type_tm.groupby(['CUS_ID']).count()
cntc_tm = cntc_tm.drop(['CNTC_SN','CNTC_DT','CNTC_CHAN_NM'], axis=1)
cntc_tm = cntc_tm.rename(columns={'CNTC_AFFR_SCN_NM':'CNTC_TM'})

In [29]:
# 전체 접촉 횟수 컬럼
cntc_w = cntc_df.groupby(['CUS_ID']).count().reset_index(drop=False)
cntc_w = cntc_w.drop(['CNTC_SN','CNTC_DT','CNTC_CHAN_NM'], axis=1)
cntc_w = cntc_w.rename(columns={'CNTC_AFFR_SCN_NM':'WHOLE_CNTC'})
cntc_w

,CUS_ID,WHOLE_CNTC
0,AONEEOO010000113,12
1,AONEEOO020000165,29
2,AONEEOO020000170,37
3,AONEEOO020000290,13
4,AONEEOO040000233,25
...,...,...
1059799,ATZZZZV310001224,18
1059800,ATZZZZV310001226,21
1059801,ATZZZZV310001228,1
1059802,ATZZZZV310001232,13


In [30]:
cntc_cnt = pd.concat([cntc_s,cntc_c,cntc_bs,cntc_e,cntc_cc,cntc_ev,
                      cntc_i,cntc_fs,cntc_td,cntc_tm], axis=1)    ### cntc_cnt : 접촉 유형별 접촉 횟수 
cntc_cnt = cntc_cnt.fillna(0)
cntc_cnt = cntc_cnt.astype(int).reset_index()
cntc_cnt = cntc_cnt.rename(columns={'index':'CUS_ID'})
cntc_cnt = cntc_cnt.sort_values(by='CUS_ID')
cntc_cnt = pd.merge(cntc_w,cntc_cnt)
cntc_cnt

,CUS_ID,WHOLE_CNTC,CNTC_SERV,CNTC_CST,CNTC_B/S,CNTC_EST,CNTC_C/C,CNTC_EVT,CNTC_INVIT,CNTC_F/S,CNTC_T/D,CNTC_TM
0,AONEEOO010000113,12,7,5,0,0,0,0,0,0,0,0
1,AONEEOO020000165,29,26,3,0,0,0,0,0,0,0,0
2,AONEEOO020000170,37,30,7,0,0,0,0,0,0,0,0
3,AONEEOO020000290,13,12,1,0,0,0,0,0,0,0,0
4,AONEEOO040000233,25,23,2,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
1059799,ATZZZZV310001224,18,18,0,0,0,0,0,0,0,0,0
1059800,ATZZZZV310001226,21,21,0,0,0,0,0,0,0,0,0
1059801,ATZZZZV310001228,1,1,0,0,0,0,0,0,0,0,0
1059802,ATZZZZV310001232,13,12,1,0,0,0,0,0,0,0,0


### [접촉정보] 접촉 유형별 횟수 컬럼을 구매유형 분류한 데이터에 결합 및 저장

In [31]:
cus_df = pd.merge(cus_df,cntc_cnt,on='CUS_ID',how='outer')
cus_df = cus_df.sort_values(by='CUS_ID').reset_index(drop=True)
cus_df = cus_df.fillna({'WHOLE_CNTC':0,'CNTC_SERV':0,'CNTC_CST':0,'CNTC_B/S':0,'CNTC_EST':0,'CNTC_C/C':0,    # 결측값은 모두 0으로 바꿔줌
                        'CNTC_EVT':0,'CNTC_INVIT':0,'CNTC_F/S':0,'CNTC_T/D':0,'CNTC_TM':0})                 
print(cus_df.info())
cus_df

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1096206 entries, 0 to 1096205
Data columns (total 22 columns):
 #   Column             Non-Null Count    Dtype  
---  ------             --------------    -----  
 0   CUS_ID             1096206 non-null  object 
 1   PSN_BIZR_YN        1096206 non-null  object 
 2   SEX_SCN_NM         1096206 non-null  object 
 3   TYMD               1096206 non-null  object 
 4   CUS_AGE            1096206 non-null  object 
 5   CUS_ADM_TRY_NM     1096206 non-null  object 
 6   CUS_N_ADMZ_NM      1096206 non-null  object 
 7   CUS_ADMB_NM        240900 non-null   object 
 8   CLB_HOUS_PYG_NM    1096206 non-null  int32  
 9   REAI_BZTC_AVG_PCE  1096206 non-null  float64
 10  PYG_AVG_PCE        1096206 non-null  float64
 11  WHOLE_CNTC         1096206 non-null  float64
 12  CNTC_SERV          1096206 non-null  float64
 13  CNTC_CST           1096206 non-null  float64
 14  CNTC_B/S           1096206 non-null  float64
 15  CNTC_EST           1096206 non-n

,CUS_ID,PSN_BIZR_YN,SEX_SCN_NM,TYMD,CUS_AGE,CUS_ADM_TRY_NM,CUS_N_ADMZ_NM,CUS_ADMB_NM,CLB_HOUS_PYG_NM,REAI_BZTC_AVG_PCE,...,CNTC_SERV,CNTC_CST,CNTC_B/S,CNTC_EST,CNTC_C/C,CNTC_EVT,CNTC_INVIT,CNTC_F/S,CNTC_T/D,CNTC_TM
0,AONEEOO010000113,N,남자,19550301,66,서울특별시,구,NaN,32,107744.0,...,7.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,AONEEOO020000165,Y,남자,19511001,70,부산광역시,구,NaN,32,38688.0,...,26.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,AONEEOO020000170,N,남자,19520401,69,NA,NA,NaN,32,0.0,...,30.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,AONEEOO020000290,Y,남자,19600701,61,인천광역시,구,NaN,32,35456.0,...,12.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,AONEEOO040000233,Y,남자,19530701,68,경기도,시,남면,32,48064.0,...,23.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1096201,ATZZZZV310001224,N,남자,19800401,41,경기도,시,NaN,32,48064.0,...,18.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1096202,ATZZZZV310001226,N,여자,19800801,41,경기도,시,NaN,32,48064.0,...,21.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1096203,ATZZZZV310001228,N,남자,19810901,40,대전광역시,구,오송읍,32,40096.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1096204,ATZZZZV310001232,N,남자,19850501,36,경기도,시,고덕면,24,10100.0,...,12.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [32]:
# 원본데이터 저장
cus_df.to_csv("'21년 해카톤_고객정보(유형별접촉횟수적용).csv", encoding='utf-8-sig', index=False)

---
---
---  
## 차량정보 전처리
---
---
---

In [33]:
### car_df(car dataframe) : car 복사본
car_df = car.copy()
car_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1835830 entries, 0 to 1835829
Data columns (total 10 columns):
 #   Column            Dtype  
---  ------            -----  
 0   CAR_ID            object 
 1   CUS_ID            object 
 2   WHOT_DT           int64  
 3   CAR_HLDG_STRT_DT  int64  
 4   CAR_HLDG_FNH_DT   float64
 5   CAR_NM            object 
 6   CAR_CGRD_NM_1     object 
 7   CAR_CGRD_NM_2     object 
 8   CAR_ENG_NM        object 
 9   CAR_TRIM_NM       object 
dtypes: float64(1), int64(2), object(7)
memory usage: 140.1+ MB


In [34]:
### car_df(car dataframe) : car 복사본
car_df = car.copy()
car_df = car_df.fillna({'CAR_HLDG_FNH_DT':19000101})                       # 결측값은 날짜형 변환을 위해 모두 '19000101'로 바꿔줌
car_df['CAR_HLDG_FNH_DT'] = car_df['CAR_HLDG_FNH_DT'].astype('int')        # 보유 종료 일자(실수형)를 날짜형으로 변환하기 위해 먼저 정수형으로 변환
car_df[['WHOT_DT','CAR_HLDG_STRT_DT','CAR_HLDG_FNH_DT']] = car_df[['WHOT_DT','CAR_HLDG_STRT_DT','CAR_HLDG_FNH_DT']].astype('str') # 정수형 데이터를 날짜형으로 변환하기 위해 문자형으로 변환
car_df['WHOT_DT'] = pd.to_datetime(car_df['WHOT_DT'])                      # Datetime 형식으로 변환
car_df['CAR_HLDG_STRT_DT'] = pd.to_datetime(car_df['CAR_HLDG_STRT_DT'])
car_df['CAR_HLDG_FNH_DT'] = pd.to_datetime(car_df['CAR_HLDG_FNH_DT'])

print(car_df.info())                                                       # 형 변환 확인
car_df.head(3)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1835830 entries, 0 to 1835829
Data columns (total 10 columns):
 #   Column            Dtype         
---  ------            -----         
 0   CAR_ID            object        
 1   CUS_ID            object        
 2   WHOT_DT           datetime64[ns]
 3   CAR_HLDG_STRT_DT  datetime64[ns]
 4   CAR_HLDG_FNH_DT   datetime64[ns]
 5   CAR_NM            object        
 6   CAR_CGRD_NM_1     object        
 7   CAR_CGRD_NM_2     object        
 8   CAR_ENG_NM        object        
 9   CAR_TRIM_NM       object        
dtypes: datetime64[ns](3), object(7)
memory usage: 140.1+ MB
None


,CAR_ID,CUS_ID,WHOT_DT,CAR_HLDG_STRT_DT,CAR_HLDG_FNH_DT,CAR_NM,CAR_CGRD_NM_1,CAR_CGRD_NM_2,CAR_ENG_NM,CAR_TRIM_NM
0,H1308052741CBFA022907,AOZZZZO010000035,2014-06-27,2014-06-27,2017-08-31,LF쏘나타,승용,중형,가솔린 2.0 CVVL,SMART
1,H1308050341LBEA656859,AOZZZZO010000067,2014-07-11,2014-07-11,1900-01-01,YF쏘나타,승용,중형,LPG 2.0,PREMIUM
2,H1308052741DBFA038433,AOZZZZO010000190,2014-07-07,2014-07-07,1900-01-01,LF쏘나타,승용,중형,LPi 2.0,PREMIUM


### [차량정보] 구매내역이 1회인 고객 정보 삭제

In [35]:
### pur_num : Purchase Number(구매 횟수)
pur_num = car_df['CUS_ID'].groupby(car_df['CUS_ID']).count()     # 각 고객별 구매 횟수를 알기 위해 groupby 실시
pur_num = pur_num.to_frame(name='PUR_NUM')                       # Series형식인 pur_num을 데이터프레임 형식으로 변경(컬럼명은 PUR_NUM으로 지정)
pur_num = pur_num.drop(pur_num[pur_num['PUR_NUM']==1].index)     # PUR_NUM 인덱스의 값이 1(구매횟수가 1회인 고객)인 row를 삭제
pur_num = pur_num.reset_index()
print(pur_num.shape)
pur_num.head()

(605770, 2)


,CUS_ID,PUR_NUM
0,AONEEOO010000113,2
1,AONEEOO020000165,2
2,AONEEOO020000170,2
3,AONEEOO020000290,2
4,AONEEOO040000233,2


In [36]:
### many_pur(Many Purchase) : 2회 이상 구매한 고객만 추출한 데이터
many_pur = pd.merge(car_df, pur_num, how='inner', on='CUS_ID')              # 구매 1회 고객을 삭제한 pur_num과 원본 데이터 car_df를 CUS_ID를 기준으로 합쳐 2회 이상 구매 고객으로 필터링
many_pur = many_pur.sort_values(by=['CUS_ID','CAR_HLDG_STRT_DT'],axis=0)    # 고객 ID, 보유 종료일 순서로 정렬
many_pur = many_pur.reset_index(drop=True)
print(many_pur.shape)
many_pur.head()

(1345394, 11)


,CAR_ID,CUS_ID,WHOT_DT,CAR_HLDG_STRT_DT,CAR_HLDG_FNH_DT,CAR_NM,CAR_CGRD_NM_1,CAR_CGRD_NM_2,CAR_ENG_NM,CAR_TRIM_NM,PUR_NUM
0,H1308031441VP6U069690,AONEEOO010000113,2006-06-27,2006-06-27,2007-07-12,베르나,승용,소형,2007이전 데이터,2007이전 데이터,2
1,H1308060741EBBA077034,AONEEOO010000113,2011-08-01,2011-08-01,1900-01-01,그랜저HG,승용,대형,가솔린 3.0,노블,2
2,H1308052141BP5A053834,AONEEOO020000165,2005-02-26,2005-02-26,1900-01-01,쏘나타,승용,중형,2007이전 데이터,2007이전 데이터,2
3,H1308070241ABAU090781,AONEEOO020000165,2010-01-06,2010-01-06,1900-01-01,제네시스,승용,대형,가솔린 3.3,Luxury,2
4,H1308190381PP1U052991,AONEEOO020000170,2001-01-11,2001-01-11,2011-06-08,싼타페,RV,중형SUV,2007이전 데이터,2007이전 데이터,2


### [차량정보] 최초구매/대차/추가구매 분류

In [37]:
# 차량 정보별 보유 종료일과 최근 보유 종료일 비교를 통해 대차/추가구매 분류
### latest_fnh : 최근 차량 보유 종료일
latest_fnh = many_pur['CAR_HLDG_FNH_DT'].groupby(many_pur['CUS_ID']).max()    # 고객별 최근 보유 종료일
latest_fnh = latest_fnh.reset_index()
latest_fnh = latest_fnh.rename(columns={'CAR_HLDG_FNH_DT':'LATEST_DT'})
print(latest_fnh.shape)
latest_fnh.head()

(605770, 2)


,CUS_ID,LATEST_DT
0,AONEEOO010000113,2007-07-12
1,AONEEOO020000165,1900-01-01
2,AONEEOO020000170,2011-06-08
3,AONEEOO020000290,2011-10-28
4,AONEEOO040000233,2017-05-30


In [38]:
# 고객ID별 구매 횟수를 나타내는 'ORD_CNT'칼럼 생성
id_list = []                # for문으로 리스트에 저장된 아이디가 없을 경우(고객별 첫 행) 1을 출력하고, 리스트에 있는 아이디의 경우 구매횟수를 +1 시킴
many_pur['ORD_CNT'] = ''    # 구매 순서 컬럼 생성
for i in range(len(many_pur)):
    if many_pur['CUS_ID'][i] not in id_list:
        j = 0
        id_list.append(many_pur['CUS_ID'][i])
        many_pur['ORD_CNT'][i] = j+1
        j+=1
    else:
        many_pur['ORD_CNT'][i] = j+1
        j+=1

<ipython-input-38-8c192f256e82>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  many_pur['ORD_CNT'][i] = j+1
<ipython-input-38-8c192f256e82>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  many_pur['ORD_CNT'][i] = j+1


In [39]:
many_pur = pd.merge(many_pur, latest_fnh, how='inner', on='CUS_ID')
print(many_pur.shape)
many_pur.head()

(1345394, 13)


,CAR_ID,CUS_ID,WHOT_DT,CAR_HLDG_STRT_DT,CAR_HLDG_FNH_DT,CAR_NM,CAR_CGRD_NM_1,CAR_CGRD_NM_2,CAR_ENG_NM,CAR_TRIM_NM,PUR_NUM,ORD_CNT,LATEST_DT
0,H1308031441VP6U069690,AONEEOO010000113,2006-06-27,2006-06-27,2007-07-12,베르나,승용,소형,2007이전 데이터,2007이전 데이터,2,1,2007-07-12
1,H1308060741EBBA077034,AONEEOO010000113,2011-08-01,2011-08-01,1900-01-01,그랜저HG,승용,대형,가솔린 3.0,노블,2,2,2007-07-12
2,H1308052141BP5A053834,AONEEOO020000165,2005-02-26,2005-02-26,1900-01-01,쏘나타,승용,중형,2007이전 데이터,2007이전 데이터,2,1,1900-01-01
3,H1308070241ABAU090781,AONEEOO020000165,2010-01-06,2010-01-06,1900-01-01,제네시스,승용,대형,가솔린 3.3,Luxury,2,2,1900-01-01
4,H1308190381PP1U052991,AONEEOO020000170,2001-01-11,2001-01-11,2011-06-08,싼타페,RV,중형SUV,2007이전 데이터,2007이전 데이터,2,1,2011-06-08


In [40]:
### 총 3가지 종류(대차, 추가구매, 최초구매)로 분류해주는 것이 목표 -> 새로운 칼럼 생성
many_pur['PUR_TYPE'] = ''                                                # PUR_TYPE : 구매 유형 저장 컬럼
for i in range(len(many_pur)):
    if many_pur['ORD_CNT'][i] == 1:                                      # 구매 순서가 1번인 경우엔 최초구매로 지정                       
        many_pur['PUR_TYPE'][i] = '최초구매'
    else:
        if many_pur['LATEST_DT'][i].year == 1900:                             # 최근 보유 종료일이 0일때는 추가구매로 분류
            many_pur['PUR_TYPE'][i] = '추가구매'
        elif many_pur['LATEST_DT'][i] == many_pur['CAR_HLDG_FNH_DT'][i]:      # 최근 보유 종료일과 보유 종료일이 같은 경우
            if many_pur['CAR_HLDG_FNH_DT'][i-1].year == 1900 or many_pur['CAR_HLDG_FNH_DT'][i-1]-many_pur['CAR_HLDG_STRT_DT'][i] >= timedelta(days=91):   # i-1행의 최근 보유 종료일이 1900년이거나, i-1행의 보유 종료일과 i행의 시작일의 차이가 91일 이상이면 '추가구매 ''
                many_pur['PUR_TYPE'][i] = '추가구매'
            else:
                many_pur['PUR_TYPE'][i] = '대차'                          # 나머지는 대차 
        elif many_pur['CAR_HLDG_FNH_DT'][i-1].year == 1900:
            many_pur['PUR_TYPE'][i] = '추가구매'
        else:                                                      
            gap = many_pur['LATEST_DT'][i]-many_pur['CAR_HLDG_STRT_DT'][i]    ### gap : 해당 아이디의 최근 보유 종료일에서 해당 아이디 i행의 보유 시작일을 뺀 값
            if gap >= timedelta(days=91):                                # 보유 시작일에서 최근 보유 종료일을 뺸 값이 91이상이면 추가구매 
                many_pur['PUR_TYPE'][i] = '추가구매'
            else:                                                        # 나머지는 대차
                many_pur['PUR_TYPE'][i] = '대차'

<ipython-input-40-9165886bbeef>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  many_pur['PUR_TYPE'][i] = '최초구매'
<ipython-input-40-9165886bbeef>:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  many_pur['PUR_TYPE'][i] = '대차'
<ipython-input-40-9165886bbeef>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  many_pur['PUR_TYPE'][i] = '추가구매'
<ipython-input-40-9165886bbeef>:19: SettingWithCopyWarning: 
A value is trying to be set on a copy

In [41]:
many_pur

,CAR_ID,CUS_ID,WHOT_DT,CAR_HLDG_STRT_DT,CAR_HLDG_FNH_DT,CAR_NM,CAR_CGRD_NM_1,CAR_CGRD_NM_2,CAR_ENG_NM,CAR_TRIM_NM,PUR_NUM,ORD_CNT,LATEST_DT,PUR_TYPE
0,H1308031441VP6U069690,AONEEOO010000113,2006-06-27,2006-06-27,2007-07-12,베르나,승용,소형,2007이전 데이터,2007이전 데이터,2,1,2007-07-12,최초구매
1,H1308060741EBBA077034,AONEEOO010000113,2011-08-01,2011-08-01,1900-01-01,그랜저HG,승용,대형,가솔린 3.0,노블,2,2,2007-07-12,대차
2,H1308052141BP5A053834,AONEEOO020000165,2005-02-26,2005-02-26,1900-01-01,쏘나타,승용,중형,2007이전 데이터,2007이전 데이터,2,1,1900-01-01,최초구매
3,H1308070241ABAU090781,AONEEOO020000165,2010-01-06,2010-01-06,1900-01-01,제네시스,승용,대형,가솔린 3.3,Luxury,2,2,1900-01-01,추가구매
4,H1308190381PP1U052991,AONEEOO020000170,2001-01-11,2001-01-11,2011-06-08,싼타페,RV,중형SUV,2007이전 데이터,2007이전 데이터,2,1,2011-06-08,최초구매
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1345389,H1308051441BP2A665825,ATZZZZV310001228,2002-04-17,2002-04-17,2004-11-09,쏘나타,승용,중형,2007이전 데이터,2007이전 데이터,2,2,2007-05-16,추가구매
1345390,H1308030751BPYU060804,ATZZZZV310001232,2000-05-31,2000-05-31,2008-04-12,베르나,승용,소형,2007이전 데이터,2007이전 데이터,2,1,2014-12-06,최초구매
1345391,H1308040741LBCU575856,ATZZZZV310001232,2012-06-15,2012-06-15,2014-12-06,아반떼MD,승용,준중형,LPI 1.6,A,2,2,2014-12-06,대차
1345392,H1308051441SPYA320981,ATZZZZV310001236,2000-06-09,2000-06-09,2004-02-04,쏘나타,승용,중형,2007이전 데이터,2007이전 데이터,2,1,2011-11-09,최초구매


### 대차 내역이 존재하는 고객의 구매정보만 따로 저장

In [42]:
c_list = []                                     # 먼저 대차 내역이 있는 CUS_ID만 따로 저장 
for i in range(len(many_pur)):
    if many_pur['PUR_TYPE'][i] == '대차':
        c_list.append(many_pur['CUS_ID'][i])
        
cus_id_list = []                                # 중복 제거 
for j in c_list:
    if j not in cus_id_list:
        cus_id_list.append(j)
cus_id_list = pd.DataFrame({'CUS_ID':cus_id_list})

### new_pur(new purchase) : 대차
new_pur = pd.merge(cus_id_list, many_pur, left_on='CUS_ID', right_on='CUS_ID', how='inner')    # CUS_ID와 나머지 데이터를 inner join해서 합치기 > 대차내역이 존재하는 고객의 구매내역만 가져오게 됨

In [43]:
# new_pur 고객 아이디와 보유 종료일을 기준으로 정렬해보기 
# 이는 아래 코드 계산을 위한 고객별 2번째로 최근인 보유 종료일을 찾기 위함임

n_temp = new_pur
n_temp.sort_values(by=['CUS_ID', 'CAR_HLDG_FNH_DT'], ascending=[True, False], inplace=True)
n_temp.reset_index(inplace=True)
n_temp.index.names=['idx']
n_temp

,index,CUS_ID,CAR_ID,WHOT_DT,CAR_HLDG_STRT_DT,CAR_HLDG_FNH_DT,CAR_NM,CAR_CGRD_NM_1,CAR_CGRD_NM_2,CAR_ENG_NM,CAR_TRIM_NM,PUR_NUM,ORD_CNT,LATEST_DT,PUR_TYPE
idx,,,,,,,,,,,,,,,
0,0,AONEEOO010000113,H1308031441VP6U069690,2006-06-27,2006-06-27,2007-07-12,베르나,승용,소형,2007이전 데이터,2007이전 데이터,2,1,2007-07-12,최초구매
1,1,AONEEOO010000113,H1308060741EBBA077034,2011-08-01,2011-08-01,1900-01-01,그랜저HG,승용,대형,가솔린 3.0,노블,2,2,2007-07-12,대차
2,2,AONEEOO020000170,H1308190381PP1U052991,2001-01-11,2001-01-11,2011-06-08,싼타페,RV,중형SUV,2007이전 데이터,2007이전 데이터,2,1,2011-06-08,최초구매
3,3,AONEEOO020000170,H1308192381UBDU081735,2012-11-20,2012-11-20,1900-01-01,싼타페 DM,RV,중형SUV,디젤 2.0,PREMIUM,2,2,2011-06-08,대차
4,4,AONEEOO020000290,H1308191081UBBU647205,2010-08-26,2010-08-26,2011-10-28,싼타페,RV,중형SUV,2.0,MLX,2,1,2011-10-28,최초구매
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
874988,874987,ATZZZZV310001199,H1308051441BPYA303010,2000-05-31,2000-05-31,2002-04-19,쏘나타,승용,중형,2007이전 데이터,2007이전 데이터,2,1,2010-02-05,최초구매
874989,874990,ATZZZZV310001232,H1308040741LBCU575856,2012-06-15,2012-06-15,2014-12-06,아반떼MD,승용,준중형,LPI 1.6,A,2,2,2014-12-06,대차
874990,874989,ATZZZZV310001232,H1308030751BPYU060804,2000-05-31,2000-05-31,2008-04-12,베르나,승용,소형,2007이전 데이터,2007이전 데이터,2,1,2014-12-06,최초구매


In [44]:
# 고객 아이디별로 몇번째 보유 종료일인지 구분하는 order칼럼을 생성해주고자 함 

idx_list = []           # 임의의 리스트 생성
n_temp['ORD_CNT'] = ''    # 새로운 칼럼 order
for i in range(len(n_temp)):
    if n_temp['CUS_ID'][i] not in idx_list:
        j = 0
        idx_list.append(n_temp['CUS_ID'][i])
        n_temp['ORD_CNT'][i] = j+1
        j+=1
    else:
        n_temp['ORD_CNT'][i] = j+1
        j+=1

<ipython-input-44-f083380c774a>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  n_temp['ORD_CNT'][i] = j+1
<ipython-input-44-f083380c774a>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  n_temp['ORD_CNT'][i] = j+1


In [45]:
temp2 = n_temp.loc[(n_temp['ORD_CNT']==2),['CUS_ID','CAR_HLDG_FNH_DT']]
temp2.rename(columns={'CAR_HLDG_FNH_DT':'LATEST_DT2'}, inplace=True)
temp2

,CUS_ID,LATEST_DT2
idx,,
1,AONEEOO010000113,1900-01-01
3,AONEEOO020000170,1900-01-01
5,AONEEOO020000290,1900-01-01
7,AONEEOO040000329,2004-01-16
9,AONEEOO040000408,2007-03-22
...,...,...
874984,ATZZZZV310001172,2006-07-26
874986,ATZZZZV310001190,1900-01-01
874988,ATZZZZV310001199,2002-04-19


In [46]:
new_pur = pd.merge(temp2, new_pur, how='inner', on='CUS_ID')
new_pur

,CUS_ID,LATEST_DT2,index,CAR_ID,WHOT_DT,CAR_HLDG_STRT_DT,CAR_HLDG_FNH_DT,CAR_NM,CAR_CGRD_NM_1,CAR_CGRD_NM_2,CAR_ENG_NM,CAR_TRIM_NM,PUR_NUM,ORD_CNT,LATEST_DT,PUR_TYPE
0,AONEEOO010000113,1900-01-01,0,H1308031441VP6U069690,2006-06-27,2006-06-27,2007-07-12,베르나,승용,소형,2007이전 데이터,2007이전 데이터,2,1,2007-07-12,최초구매
1,AONEEOO010000113,1900-01-01,1,H1308060741EBBA077034,2011-08-01,2011-08-01,1900-01-01,그랜저HG,승용,대형,가솔린 3.0,노블,2,2,2007-07-12,대차
2,AONEEOO020000170,1900-01-01,2,H1308190381PP1U052991,2001-01-11,2001-01-11,2011-06-08,싼타페,RV,중형SUV,2007이전 데이터,2007이전 데이터,2,1,2011-06-08,최초구매
3,AONEEOO020000170,1900-01-01,3,H1308192381UBDU081735,2012-11-20,2012-11-20,1900-01-01,싼타페 DM,RV,중형SUV,디젤 2.0,PREMIUM,2,2,2011-06-08,대차
4,AONEEOO020000290,1900-01-01,4,H1308191081UBBU647205,2010-08-26,2010-08-26,2011-10-28,싼타페,RV,중형SUV,2.0,MLX,2,1,2011-10-28,최초구매
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
874988,ATZZZZV310001199,2002-04-19,874987,H1308051441BPYA303010,2000-05-31,2000-05-31,2002-04-19,쏘나타,승용,중형,2007이전 데이터,2007이전 데이터,2,2,2010-02-05,최초구매
874989,ATZZZZV310001232,2008-04-12,874990,H1308040741LBCU575856,2012-06-15,2012-06-15,2014-12-06,아반떼MD,승용,준중형,LPI 1.6,A,2,1,2014-12-06,대차
874990,ATZZZZV310001232,2008-04-12,874989,H1308030751BPYU060804,2000-05-31,2000-05-31,2008-04-12,베르나,승용,소형,2007이전 데이터,2007이전 데이터,2,2,2014-12-06,최초구매
874991,ATZZZZV310001236,2004-02-04,874992,H1308060841NBBA056058,2011-06-10,2011-06-10,2011-11-09,그랜저HG,승용,대형,LPG 3.0,PRIME,2,1,2011-11-09,대차


In [47]:
from datetime import timedelta
new_pur['LATEST_DT'] = pd.to_datetime(new_pur['LATEST_DT'])
new_pur['LATEST_DT2'] = pd.to_datetime(new_pur['LATEST_DT2'])
new_pur['CAR_HLDG_STRT_DT'] = pd.to_datetime(new_pur['CAR_HLDG_STRT_DT'])
new_pur['CAR_HLDG_FNH_DT'] = pd.to_datetime(new_pur['CAR_HLDG_FNH_DT'])

new_pur['ET'] = ''    ### ET(Elapsed Time : 경과시간)
for i in range(len(new_pur)):
    if new_pur['PUR_TYPE'][i] == '최초구매':
        pass
    else:
        if new_pur['PUR_TYPE'][i+1] == '최초구매':
            if new_pur['LATEST_DT'][i] != new_pur['CAR_HLDG_FNH_DT'][i]:
                gap1 = new_pur['CAR_HLDG_STRT_DT'][i]-new_pur['LATEST_DT'][i]
                if gap1 >= timedelta(days=183):
                    new_pur['ET'][i] = '6개월이상'
                else:
                    new_pur['ET'][i] = '6개월미만'
            else:
                new_pur['LATEST_DT'][i] == new_pur['CAR_HLDG_FNH_DT'][i]
                gap2 = new_pur['CAR_HLDG_STRT_DT'][i]-new_pur['LATEST_DT2'][i]
                if gap2 >= timedelta(days=183):
                    new_pur['ET'][i] = '6개월이상'
                else:
                    new_pur['ET'][i] = '6개월미만'
        else: 
            pass

<ipython-input-47-748b7b982bb1>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_pur['ET'][i] = '6개월이상'
<ipython-input-47-748b7b982bb1>:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_pur['ET'][i] = '6개월이상'
<ipython-input-47-748b7b982bb1>:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_pur['ET'][i] = '6개월미만'
<ipython-input-47-748b7b982bb1>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice fr

In [48]:
new_pur.sort_values(by=['CUS_ID','CAR_HLDG_STRT_DT'],axis=0)

,CUS_ID,LATEST_DT2,index,CAR_ID,WHOT_DT,CAR_HLDG_STRT_DT,CAR_HLDG_FNH_DT,CAR_NM,CAR_CGRD_NM_1,CAR_CGRD_NM_2,CAR_ENG_NM,CAR_TRIM_NM,PUR_NUM,ORD_CNT,LATEST_DT,PUR_TYPE,ET
0,AONEEOO010000113,1900-01-01,0,H1308031441VP6U069690,2006-06-27,2006-06-27,2007-07-12,베르나,승용,소형,2007이전 데이터,2007이전 데이터,2,1,2007-07-12,최초구매,
1,AONEEOO010000113,1900-01-01,1,H1308060741EBBA077034,2011-08-01,2011-08-01,1900-01-01,그랜저HG,승용,대형,가솔린 3.0,노블,2,2,2007-07-12,대차,6개월이상
2,AONEEOO020000170,1900-01-01,2,H1308190381PP1U052991,2001-01-11,2001-01-11,2011-06-08,싼타페,RV,중형SUV,2007이전 데이터,2007이전 데이터,2,1,2011-06-08,최초구매,
3,AONEEOO020000170,1900-01-01,3,H1308192381UBDU081735,2012-11-20,2012-11-20,1900-01-01,싼타페 DM,RV,중형SUV,디젤 2.0,PREMIUM,2,2,2011-06-08,대차,6개월이상
4,AONEEOO020000290,1900-01-01,4,H1308191081UBBU647205,2010-08-26,2010-08-26,2011-10-28,싼타페,RV,중형SUV,2.0,MLX,2,1,2011-10-28,최초구매,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
874987,ATZZZZV310001199,2002-04-19,874988,H1308062141CP3A296722,2003-06-23,2003-06-23,2010-02-05,그랜저,승용,대형,2007이전 데이터,2007이전 데이터,2,1,2010-02-05,대차,6개월이상
874990,ATZZZZV310001232,2008-04-12,874989,H1308030751BPYU060804,2000-05-31,2000-05-31,2008-04-12,베르나,승용,소형,2007이전 데이터,2007이전 데이터,2,2,2014-12-06,최초구매,
874989,ATZZZZV310001232,2008-04-12,874990,H1308040741LBCU575856,2012-06-15,2012-06-15,2014-12-06,아반떼MD,승용,준중형,LPI 1.6,A,2,1,2014-12-06,대차,6개월이상
874992,ATZZZZV310001236,2004-02-04,874991,H1308051441SPYA320981,2000-06-09,2000-06-09,2004-02-04,쏘나타,승용,중형,2007이전 데이터,2007이전 데이터,2,2,2011-11-09,최초구매,


In [62]:
final_res = new_pur.loc[(new_pur['ET']=='6개월이상')|(new_pur['ET']=='6개월미만')]
final_res

,CUS_ID,LATEST_DT2,index,CAR_ID,WHOT_DT,CAR_HLDG_STRT_DT,CAR_HLDG_FNH_DT,CAR_NM,CAR_CGRD_NM_1,CAR_CGRD_NM_2,CAR_ENG_NM,CAR_TRIM_NM,PUR_NUM,ORD_CNT,LATEST_DT,PUR_TYPE,ET
1,AONEEOO010000113,1900-01-01,1,H1308060741EBBA077034,2011-08-01,2011-08-01,1900-01-01,그랜저HG,승용,대형,가솔린 3.0,노블,2,2,2007-07-12,대차,6개월이상
3,AONEEOO020000170,1900-01-01,3,H1308192381UBDU081735,2012-11-20,2012-11-20,1900-01-01,싼타페 DM,RV,중형SUV,디젤 2.0,PREMIUM,2,2,2011-06-08,대차,6개월이상
6,AONEEOO040000329,2004-01-16,7,H1308042041BP7U177233,2007-03-15,2007-03-15,2011-11-26,아반떼,승용,준중형,가솔린 1.6,E16,2,1,2011-11-26,대차,6개월이상
8,AONEEOO040000408,2007-03-22,9,H1308060141BP9A344364,2008-04-08,2008-04-08,2009-09-26,그랜저,승용,대형,가솔린 Q270,디럭스,2,1,2009-09-26,대차,6개월이상
10,AONEEOO050000406,2004-02-04,11,H1308060141BP8A266545,2007-07-16,2007-07-16,2007-07-18,그랜저,승용,대형,가솔린 Q270,디럭스,3,1,2007-07-18,대차,6개월이상
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
874981,ATZZZZV310001170,2006-12-27,874982,H1308032051UBDU104344,2013-02-21,2013-02-21,2014-07-24,엑센트,승용,소형,디젤 1.6,Premier,2,1,2014-07-24,대차,6개월이상
874983,ATZZZZV310001172,2006-07-26,874984,H1308042141TP7U199924,2007-04-09,2007-04-09,2010-01-29,아반떼,승용,준중형,해당없음,해당없음,2,1,2010-01-29,대차,6개월이상
874987,ATZZZZV310001199,2002-04-19,874988,H1308062141CP3A296722,2003-06-23,2003-06-23,2010-02-05,그랜저,승용,대형,2007이전 데이터,2007이전 데이터,2,1,2010-02-05,대차,6개월이상
874989,ATZZZZV310001232,2008-04-12,874990,H1308040741LBCU575856,2012-06-15,2012-06-15,2014-12-06,아반떼MD,승용,준중형,LPI 1.6,A,2,1,2014-12-06,대차,6개월이상


In [6]:
del final_res['index']
final_res = final_res[['CUS_ID', 'CAR_ID', 'WHOT_DT', 'CAR_HLDG_STRT_DT','CAR_HLDG_FNH_DT', 'CAR_NM', 'CAR_CGRD_NM_1','CAR_CGRD_NM_2',
                       'CAR_ENG_NM', 'CAR_TRIM_NM', 'PUR_NUM', 'ORD_CNT', 'LATEST_DT', 'LATEST_DT2','PUR_TYPE', 'ET']]
final_res = final_res.reset_index(drop=True)
final_res

,CUS_ID,CAR_ID,WHOT_DT,CAR_HLDG_STRT_DT,CAR_HLDG_FNH_DT,CAR_NM,CAR_CGRD_NM_1,CAR_CGRD_NM_2,CAR_ENG_NM,CAR_TRIM_NM,PUR_NUM,ORD_CNT,LATEST_DT,LATEST_DT2,PUR_TYPE,ET
0,AONEEOO010000113,H1308060741EBBA077034,2011-08-01,2011-08-01,1900-01-01,그랜저HG,승용,대형,가솔린 3.0,노블,2,2,2007-07-12,1900-01-01,대차,6개월이상
1,AONEEOO020000170,H1308192381UBDU081735,2012-11-20,2012-11-20,1900-01-01,싼타페 DM,RV,중형SUV,디젤 2.0,PREMIUM,2,2,2011-06-08,1900-01-01,대차,6개월이상
2,AONEEOO040000329,H1308042041BP7U177233,2007-03-15,2007-03-15,2011-11-26,아반떼,승용,준중형,가솔린 1.6,E16,2,1,2011-11-26,2004-01-16,대차,6개월이상
3,AONEEOO040000408,H1308060141BP9A344364,2008-04-08,2008-04-08,2009-09-26,그랜저,승용,대형,가솔린 Q270,디럭스,2,1,2009-09-26,2007-03-22,대차,6개월이상
4,AONEEOO050000406,H1308060141BP8A266545,2007-07-16,2007-07-16,2007-07-18,그랜저,승용,대형,가솔린 Q270,디럭스,3,1,2007-07-18,2004-02-04,대차,6개월이상
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
331739,ATZZZZV310001170,H1308032051UBDU104344,2013-02-21,2013-02-21,2014-07-24,엑센트,승용,소형,디젤 1.6,Premier,2,1,2014-07-24,2006-12-27,대차,6개월이상
331740,ATZZZZV310001172,H1308042141TP7U199924,2007-04-09,2007-04-09,2010-01-29,아반떼,승용,준중형,해당없음,해당없음,2,1,2010-01-29,2006-07-26,대차,6개월이상
331741,ATZZZZV310001199,H1308062141CP3A296722,2003-06-23,2003-06-23,2010-02-05,그랜저,승용,대형,2007이전 데이터,2007이전 데이터,2,1,2010-02-05,2002-04-19,대차,6개월이상
331742,ATZZZZV310001232,H1308040741LBCU575856,2012-06-15,2012-06-15,2014-12-06,아반떼MD,승용,준중형,LPI 1.6,A,2,1,2014-12-06,2008-04-12,대차,6개월이상


### [차량정보] 최초구매/대차/추가구매, 구매 경과 6개월내외 분류한 데이터 저장

In [7]:
####원본데이터 저장
final_res.to_csv("'21년 해카톤_차량정보(구매유형분류,경과시간적용).csv", encoding='utf-8-sig', index=False)

## 분석용 데이터 생성

In [65]:
data_set = pd.merge(cus_df, final_res, on='CUS_ID')
data_set.info()
data_set.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 331744 entries, 0 to 331743
Data columns (total 38 columns):
 #   Column             Non-Null Count   Dtype         
---  ------             --------------   -----         
 0   CUS_ID             331744 non-null  object        
 1   PSN_BIZR_YN        331744 non-null  object        
 2   SEX_SCN_NM         331744 non-null  object        
 3   TYMD               331744 non-null  object        
 4   CUS_AGE            331744 non-null  object        
 5   CUS_ADM_TRY_NM     331744 non-null  object        
 6   CUS_N_ADMZ_NM      331744 non-null  object        
 7   CUS_ADMB_NM        78507 non-null   object        
 8   CLB_HOUS_PYG_NM    331744 non-null  int32         
 9   REAI_BZTC_AVG_PCE  331744 non-null  float64       
 10  PYG_AVG_PCE        331744 non-null  float64       
 11  WHOLE_CNTC         331744 non-null  float64       
 12  CNTC_SERV          331744 non-null  float64       
 13  CNTC_CST           331744 non-null  float64 

,CUS_ID,PSN_BIZR_YN,SEX_SCN_NM,TYMD,CUS_AGE,CUS_ADM_TRY_NM,CUS_N_ADMZ_NM,CUS_ADMB_NM,CLB_HOUS_PYG_NM,REAI_BZTC_AVG_PCE,...,CAR_NM,CAR_CGRD_NM_1,CAR_CGRD_NM_2,CAR_ENG_NM,CAR_TRIM_NM,PUR_NUM,ORD_CNT,LATEST_DT,PUR_TYPE,ET
0,AONEEOO010000113,N,남자,19550301,66,서울특별시,구,NaN,32,107744.0,...,그랜저HG,승용,대형,가솔린 3.0,노블,2,2,2007-07-12,대차,6개월이상
1,AONEEOO020000170,N,남자,19520401,69,NA,NA,NaN,32,0.0,...,싼타페 DM,RV,중형SUV,디젤 2.0,PREMIUM,2,2,2011-06-08,대차,6개월이상
2,AONEEOO040000329,N,남자,19580401,63,경상북도,군,성산면,32,18016.0,...,아반떼,승용,준중형,가솔린 1.6,E16,2,1,2011-11-26,대차,6개월이상
3,AONEEOO040000408,Y,남자,19631001,58,전라남도,군,해제면,32,20192.0,...,그랜저,승용,대형,가솔린 Q270,디럭스,2,1,2009-09-26,대차,6개월이상
4,AONEEOO050000406,Y,남자,19641201,57,서울특별시,구,NaN,34,130000.0,...,그랜저,승용,대형,가솔린 Q270,디럭스,3,1,2007-07-18,대차,6개월이상


In [24]:
### dataset : 최종 분석 데이터셋
dataset = data_set.drop(['CUS_ID','TYMD','CUS_ADMB_NM','PYG_AVG_PCE','CAR_ID','WHOT_DT','CAR_HLDG_STRT_DT','CAR_HLDG_FNH_DT',
                         'CAR_NM','CAR_ENG_NM','CAR_TRIM_NM','ORD_CNT','LATEST_DT','LATEST_DT2'], axis=1)
dataset = dataset.dropna(axis=0)                            #시군구 결측치 삭제
del_na = dataset[dataset['CUS_N_ADMZ_NM'] == 'NA'].index    ### del_ap(delete add purchase)
dataset = dataset.drop(del_na).reset_index(drop=True)       #추가구매인 행 삭제
del_ap = dataset[dataset['PUR_TYPE'] == '추가구매'].index    ### del_ap(delete add purchase)
dataset = dataset.drop(del_ap).reset_index(drop=True)       #추가구매인 행 삭제
dataset = dataset.drop(['PUR_TYPE'], axis=1)

In [25]:
dataset.info()
dataset.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 250609 entries, 0 to 250608
Data columns (total 22 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   PSN_BIZR_YN        250609 non-null  object 
 1   SEX_SCN_NM         250609 non-null  object 
 2   CUS_AGE            250609 non-null  int64  
 3   CUS_ADM_TRY_NM     250609 non-null  object 
 4   CUS_N_ADMZ_NM      250609 non-null  object 
 5   CLB_HOUS_PYG_NM    250609 non-null  int64  
 6   REAI_BZTC_AVG_PCE  250609 non-null  float64
 7   WHOLE_CNTC         250609 non-null  float64
 8   CNTC_SERV          250609 non-null  float64
 9   CNTC_CST           250609 non-null  float64
 10  CNTC_B/S           250609 non-null  float64
 11  CNTC_EST           250609 non-null  float64
 12  CNTC_C/C           250609 non-null  float64
 13  CNTC_EVT           250609 non-null  float64
 14  CNTC_INVIT         250609 non-null  float64
 15  CNTC_F/S           250609 non-null  float64
 16  CN

,PSN_BIZR_YN,SEX_SCN_NM,CUS_AGE,CUS_ADM_TRY_NM,CUS_N_ADMZ_NM,CLB_HOUS_PYG_NM,REAI_BZTC_AVG_PCE,WHOLE_CNTC,CNTC_SERV,CNTC_CST,CNTC_B/S,CNTC_EST,CNTC_C/C,CNTC_EVT,CNTC_INVIT,CNTC_F/S,CNTC_T/D,CNTC_TM,CAR_CGRD_NM_1,CAR_CGRD_NM_2,PUR_NUM,ET
0,N,남자,66,서울특별시,구,32,107744.0,12.0,7.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,승용,대형,2,6개월이상
1,N,남자,63,경상북도,군,32,18016.0,2.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,승용,준중형,2,6개월이상
2,Y,남자,58,전라남도,군,32,20192.0,17.0,15.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,승용,대형,2,6개월이상
3,Y,남자,57,서울특별시,구,34,130000.0,21.0,15.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,승용,대형,3,6개월이상
4,N,남자,60,서울특별시,구,21,70707.0,10.0,5.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,승용,대형,3,6개월미만


### [분석용 데이터] 데이터 저장

In [26]:
dataset.to_csv("'21년 해카톤_분석데이터셋.csv", encoding='utf-8-sig', index=False)